In [2]:
import pandas as pd
import numpy as np

In [46]:
df_index= pd.read_excel('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/CSI300_Index_Level.xlsx')
df_index.head()

,date,index_level
0,2010-04-16,3356.332
1,2010-04-19,3176.423
2,2010-04-20,3173.374
3,2010-04-21,3236.679
4,2010-04-22,3201.541


In [47]:
df_2008 = pd.read_excel('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/CSI300_Index_Level(2008-3-31).xlsx')
df_2008.head()



,指标名称,沪深300指数
0,频率,日
1,单位,点
2,指标ID,M002845714
3,2022-12-30,3871.63
4,2022-12-29,3856.7


In [48]:
a = ['2010-04-15',3394.57]
df_index2 = pd.DataFrame(np.insert(df_index.values,0,a,axis = 0))
df_index2.columns = df_index.columns
df_index2

,date,index_level
0,2010-04-15,3394.57
1,2010-04-16,3356.332
2,2010-04-19,3176.423
3,2010-04-20,3173.374
4,2010-04-21,3236.679
...,...,...
3087,2022-12-26,3843.4887
3088,2022-12-27,3887.854
3089,2022-12-28,3871.2644
3090,2022-12-29,3856.7007


In [49]:
diff = np.array(df_index2["index_level"])
diff = np.diff(diff)
diff

array([-38.238000000000284, -179.9090000000001, -3.048999999999978, ...,
       -16.58959999999979, -14.563700000000154, 14.933100000000195],
      dtype=object)

In [50]:
df_index3 = pd.DataFrame(np.insert(df_index.values,2,diff,axis = 1))
df_index3.columns = ['date','index_level','diff']
df_index3['date']=pd.to_datetime(df_index3['date'])
df_index3

,date,index_level,diff
0,2010-04-16,3356.332,-38.238
1,2010-04-19,3176.423,-179.909
2,2010-04-20,3173.374,-3.049
3,2010-04-21,3236.679,63.305
4,2010-04-22,3201.541,-35.138
...,...,...,...
3086,2022-12-26,3843.4887,15.2699
3087,2022-12-27,3887.854,44.3653
3088,2022-12-28,3871.2644,-16.5896
3089,2022-12-29,3856.7007,-14.5637


In [51]:
df_index3.set_index("date", inplace=True)
df_index3


,index_level,diff
date,,
2010-04-16,3356.332,-38.238
2010-04-19,3176.423,-179.909
2010-04-20,3173.374,-3.049
2010-04-21,3236.679,63.305
2010-04-22,3201.541,-35.138
...,...,...
2022-12-26,3843.4887,15.2699
2022-12-27,3887.854,44.3653
2022-12-28,3871.2644,-16.5896


In [61]:
df_index4 = df_index3.drop('index_level',axis = 1)
df_index4
#check missing value
df_index4.isnull().sum()
#find out missing value
df_index4[df_index4.isnull().values==True]
df_index4.dropna(inplace=True)
df_index4

,diff
date,
2010-04-16,-38.238
2010-04-19,-179.909
2010-04-20,-3.049
2010-04-21,63.305
2010-04-22,-35.138
...,...
2022-12-26,15.2699
2022-12-27,44.3653
2022-12-28,-16.5896


In [53]:
df_index4['binary'] = df_index4['diff'].apply(lambda x: 1 if x > 0 else 0)
df_index4

,diff,binary
date,,
2010-04-16,-38.238,0
2010-04-19,-179.909,0
2010-04-20,-3.049,0
2010-04-21,63.305,1
2010-04-22,-35.138,0
...,...,...
2022-12-26,15.2699,1
2022-12-27,44.3653,1
2022-12-28,-16.5896,0


In [54]:
weekly_groups=df_index4.groupby([pd.Grouper(freq='W'),'binary']).mean()
weekly_groups

diff
date       binary           
2010-04-18 0      -38.238000
2010-04-25 0      -57.408500
           1       63.305000
2010-05-02 0      -32.484750
           1        7.301000
...                      ...
2022-12-18 1        5.721450
2022-12-25 0      -44.339633
           1        3.505350
2023-01-01 0      -15.576650
           1       24.856100

[1245 rows x 1 columns]

In [55]:
df = weekly_groups
df

diff
date       binary           
2010-04-18 0      -38.238000
2010-04-25 0      -57.408500
           1       63.305000
2010-05-02 0      -32.484750
           1        7.301000
...                      ...
2022-12-18 1        5.721450
2022-12-25 0      -44.339633
           1        3.505350
2023-01-01 0      -15.576650
           1       24.856100

[1245 rows x 1 columns]

In [56]:
grouped = df.groupby('date')['diff']
diff1 = grouped.apply(lambda x: x.loc[x.index.get_level_values('binary') == 1].sum() / x.loc[x.index.get_level_values('binary') == 0].sum() if 0 in x.index.get_level_values('binary') and 1 in x.index.get_level_values('binary') else '+' if 1 in x.index.get_level_values('binary') else '-').to_numpy()
diff1

array(['-', -1.1027112709790412, -0.22475161421897702,
       -0.20537651350794184, -0.9395575592367362, -0.7506418840681859,
       -1.4519023741745352, -0.25487691138089946, -0.7861180567590093,
       '-', -2.768768578837507, -0.15287133234387695, -3.562962962962933,
       -0.4285225547290964, '+', -2.7317226060531907, -0.916425039115492,
       -0.4315905817731179, -1.1267979760502875, -0.2929781285972684,
       -4.375310983953198, -0.8119968930941972, -0.22405645712864466,
       -0.6626808768737568, -3.584571598768895, '+', '+',
       -1.4439691653132662, -4.052543077014381, -2.449563819146372,
       -0.21654951288691632, -0.3096557291839464, -1.7393844527970816,
       -0.5643135188427475, -0.6997605220878309, -3.6803117796087936,
       -2.1219844561913135, -0.4706952258297748, -2.2771064283098683,
       -0.4703338475099183, -0.3346167394134316, -1.459473535407643, '+',
       -1.100993559751203, -5.077367993075045, -0.21220979539913065,
       -2.1346194423511595, -0.4969

In [57]:
diff2 = list(diff1)
diff2

['-',
 -1.1027112709790412,
 -0.22475161421897702,
 -0.20537651350794184,
 -0.9395575592367362,
 -0.7506418840681859,
 -1.4519023741745352,
 -0.25487691138089946,
 -0.7861180567590093,
 '-',
 -2.768768578837507,
 -0.15287133234387695,
 -3.562962962962933,
 -0.4285225547290964,
 '+',
 -2.7317226060531907,
 -0.916425039115492,
 -0.4315905817731179,
 -1.1267979760502875,
 -0.2929781285972684,
 -4.375310983953198,
 -0.8119968930941972,
 -0.22405645712864466,
 -0.6626808768737568,
 -3.584571598768895,
 '+',
 '+',
 -1.4439691653132662,
 -4.052543077014381,
 -2.449563819146372,
 -0.21654951288691632,
 -0.3096557291839464,
 -1.7393844527970816,
 -0.5643135188427475,
 -0.6997605220878309,
 -3.6803117796087936,
 -2.1219844561913135,
 -0.4706952258297748,
 -2.2771064283098683,
 -0.4703338475099183,
 -0.3346167394134316,
 -1.459473535407643,
 '+',
 -1.100993559751203,
 -5.077367993075045,
 -0.21220979539913065,
 -2.1346194423511595,
 -0.49694293230726644,
 -0.47469424100895863,
 -3.039415838602843

In [58]:
diff3 = []
for i in diff2:
    if i != '+' and i != '-':
        i = abs(float(i))
        i = 100*(1-1/(1+i))
    diff3.append(i)
diff3

['-',
 52.442353175079944,
 18.35079142657843,
 17.03837026907432,
 48.441849779723746,
 42.87809465199275,
 59.21534191031309,
 20.310909306668677,
 44.01265939752356,
 '-',
 73.46613412096387,
 13.260051495345781,
 78.08441558441544,
 29.997605099792146,
 '+',
 73.20272416878173,
 47.81950874209301,
 30.147626511943436,
 52.98095958050906,
 22.659171266502064,
 81.39642519316038,
 44.812267404477566,
 18.304421811901527,
 39.85616759601863,
 78.18771114255185,
 '+',
 '+',
 59.08295349250772,
 80.20798665627777,
 71.01082767480267,
 17.80030410542326,
 23.64405563108517,
 63.49544880497011,
 36.07419561650384,
 41.1681829878193,
 78.63390203283454,
 67.96909100502195,
 32.004946882465454,
 69.48527544417473,
 31.98823507371823,
 25.07212217048145,
 59.34089204036683,
 '+',
 52.403471426232315,
 83.54550849743727,
 17.50602875876438,
 68.09820080583889,
 33.19718618406641,
 32.18933307043916,
 75.24394516545043,
 73.52914931588334,
 '+',
 62.76936805922782,
 29.016843238418176,
 47.255

In [59]:
diff4 = []
for i in diff3:
    if i == '-':
        i = int('0')
    if i =='+':
        i = int('100')
    diff4.append(i)
diff4

[0,
 52.442353175079944,
 18.35079142657843,
 17.03837026907432,
 48.441849779723746,
 42.87809465199275,
 59.21534191031309,
 20.310909306668677,
 44.01265939752356,
 0,
 73.46613412096387,
 13.260051495345781,
 78.08441558441544,
 29.997605099792146,
 100,
 73.20272416878173,
 47.81950874209301,
 30.147626511943436,
 52.98095958050906,
 22.659171266502064,
 81.39642519316038,
 44.812267404477566,
 18.304421811901527,
 39.85616759601863,
 78.18771114255185,
 100,
 100,
 59.08295349250772,
 80.20798665627777,
 71.01082767480267,
 17.80030410542326,
 23.64405563108517,
 63.49544880497011,
 36.07419561650384,
 41.1681829878193,
 78.63390203283454,
 67.96909100502195,
 32.004946882465454,
 69.48527544417473,
 31.98823507371823,
 25.07212217048145,
 59.34089204036683,
 100,
 52.403471426232315,
 83.54550849743727,
 17.50602875876438,
 68.09820080583889,
 33.19718618406641,
 32.18933307043916,
 75.24394516545043,
 73.52914931588334,
 100,
 62.76936805922782,
 29.016843238418176,
 47.2556250

In [64]:
df_index5=df_index4.groupby([pd.Grouper(freq='W')]).mean()
df_index5
df_index5.dropna(inplace=True)
df_index5['RSI'] = diff4
df_index5 = df_index5.drop('diff',axis = 1)
df_index5

,RSI
date,
2010-04-18,0.000000
2010-04-25,52.442353
2010-05-02,18.350791
2010-05-09,17.038370
2010-05-16,48.441850
...,...
2022-12-04,61.873567
2022-12-11,77.835251
2022-12-18,23.634396


In [65]:
df_index5.to_csv('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/weekly_RSI.csv', index = False, header=True)